In [1]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd

from calenviroscreen_utils import *
from utils import *
import prep_data

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:68: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
ces_df = prep_data.generate_calenviroscreen_lehd_data(prep_data.datasets)

In [3]:
ces_df

,Tract,ZIP,Population,sq_mi,pop_sq_mi,overall_ptile,pollution_ptile,popchar_ptile,equity_group,pollution_group,popchar_group,County,City,geometry,num_jobs,jobs_sq_mi
0,06001400100,94704,3120,2.655917,1174.735658,2.798790,26.621033,1.525466,1,1,1,Alameda,Oakland,"POLYGON ((-197090.096 -12468.283, -196909.112 ...",936,352.420697
1,06001400200,94618,2007,0.229901,8729.842746,2.874433,24.181705,1.651538,1,1,1,Alameda,Oakland,"POLYGON ((-196982.196 -15963.566, -196992.931 ...",1357,5902.539415
2,06001400300,94618,5051,0.427356,11819.185813,15.935451,33.366521,12.266768,1,2,1,Alameda,Oakland,"POLYGON ((-197350.929 -16712.642, -197950.200 ...",1978,4628.459619
3,06001400400,94609,4007,0.271558,14755.587549,18.973777,26.235221,18.431669,1,1,1,Alameda,Oakland,"POLYGON ((-197953.290 -16012.154, -197963.187 ...",983,3619.850901
4,06001400500,94609,4124,0.227012,18166.435207,29.740292,31.400124,30.156329,1,1,1,Alameda,Oakland,"POLYGON ((-198589.270 -15822.210, -198703.191 ...",362,1594.628891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8030,06115040800,95692,4652,42.061128,110.600934,41.275845,48.263846,36.220373,2,2,2,Yuba,Unincorporated Yuba County area,"POLYGON ((-118271.694 119288.741, -117977.343 ...",876,20.826831
8031,06115040901,95901,2720,100.002680,27.199271,59.127584,63.634101,50.642965,2,2,2,Yuba,Unincorporated Yuba County area,"POLYGON ((-112857.633 134628.595, -112795.165 ...",849,8.489772
8032,06115040902,95901,1653,39.586578,41.756577,34.430156,23.795893,42.423096,2,1,2,Yuba,Unincorporated Yuba County area,"POLYGON ((-114108.120 127146.968, -114190.344 ...",569,14.373559
8033,06115041000,95901,7298,183.593917,39.750772,42.133132,59.352831,31.732224,2,2,1,Yuba,Unincorporated Yuba County area,"POLYGON ((-118015.890 150723.763, -118020.386 ...",741,4.036081


In [4]:
ces_df['tract_type'] = ces_df['pop_sq_mi'].apply(lambda x: 'urban' if x > 2400 else 'suburban' if x > 800 else 'rural')

In [5]:
with_funding = pd.read_parquet(f"{GCS_FILE_PATH}shape_frequency_funding.parquet")

In [6]:
with_funding

,calitp_itp_id,ntd_id,transit_provider,_5307_funds,_5311_funds,operating_expenses_total_2019,shape_id,day_name,departure_hour,route_id,trips_per_hour,mean_runtime_min
0,257,None,PresidioGo Shuttle,0.0,NaN,0.0,13737,Thursday,0,66,0,NaN
1,257,None,PresidioGo Shuttle,0.0,NaN,0.0,13737,Thursday,1,66,0,NaN
2,257,None,PresidioGo Shuttle,0.0,NaN,0.0,13737,Thursday,2,66,0,NaN
3,257,None,PresidioGo Shuttle,0.0,NaN,0.0,13737,Thursday,3,66,0,NaN
4,257,None,PresidioGo Shuttle,0.0,NaN,0.0,13737,Thursday,4,66,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
423211,251,90196,Placer County Transit,2027385.0,405065.0,13746322.0,p_298772,Sunday,19,11117,0,NaN
423212,251,90196,Placer County Transit,2027385.0,405065.0,13746322.0,p_298772,Sunday,20,11117,0,NaN
423213,251,90196,Placer County Transit,2027385.0,405065.0,13746322.0,p_298772,Sunday,21,11117,0,NaN
423214,251,90196,Placer County Transit,2027385.0,405065.0,13746322.0,p_298772,Sunday,22,11117,0,NaN


In [7]:
dates = get_recent_dates()
min_date = min(dates.values())
max_date = max(dates.values())

In [8]:
bbb_shapes = (tbl.gtfs_schedule.shapes() 
          >> filter(_.calitp_itp_id == 300)
          >> collect()
         )

In [10]:
bbb_shapes = gpd.GeoDataFrame(bbb_shapes, 
                          geometry = gpd.points_from_xy(bbb_shapes.shape_pt_lon, bbb_shapes.shape_pt_lat),
                          crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)

In [11]:
bbb_shapes

,calitp_itp_id,calitp_url_number,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,calitp_extracted_at,geometry
0,300,0,25386,34.01593,-118.476129,89,5.0062,2021-10-21,POINT (140748.035 -443268.809)
1,300,0,25387,34.013909,-118.468349,33,2.3285,2021-10-21,POINT (141470.171 -443481.378)
2,300,0,25387,34.018,-118.44438,91,5.6137,2021-10-21,POINT (143676.413 -442991.602)
3,300,0,25387,34.02204,-118.44515,83,5.1542,2021-10-21,POINT (143597.954 -442544.693)
4,300,0,25387,34.02404,-118.44717,72,4.7694,2021-10-21,POINT (143407.780 -442325.930)
...,...,...,...,...,...,...,...,...,...
22213,300,0,25403,34.046109,-118.52429,123,2.98,2021-10-21,POINT (136248.041 -439991.941)
22214,300,0,25403,34.04148,-118.51863,134,3.7133,2021-10-21,POINT (136778.592 -440497.213)
22215,300,0,25403,34.04013,-118.518109,141,3.8769,2021-10-21,POINT (136829.034 -440646.195)
22216,300,0,25403,34.0485,-118.541879,41,1.1247,2021-10-21,POINT (134620.149 -439751.869)


In [26]:
def shape_line(df):

    df.shape_pt_sequence = df.shape_pt_sequence.astype('int64')
    df.sort_values(by='shape_pt_sequence', inplace=True) ##arrange, then convert to line to preserve order...
         
    route_line = LineString(list(df['geometry']))
    df['route_line'] = route_line
    # df['geometry'] = route_line
    return df
    # single_line = single_shape[['calitp_itp_id', 'shape_id', 'calitp_extracted_at']].iloc[[0]] ##preserve info cols
    # single_line['geometry'] = route_line
    # single_line = gpd.GeoDataFrame(single_line, crs='EPSG:6414')

In [27]:
import shapely
from shapely.geometry import LineString

In [30]:
bbb_shapes.groupby('shape_id').apply(shape_line).reset_index(drop=True)

,calitp_itp_id,calitp_url_number,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,calitp_extracted_at,geometry,route_line
0,300,0,25311,34.01402,-118.492379,1,0,2021-10-21,POINT (139250.645 -443504.643),LINESTRING (139250.6453259104 -443504.64285780...
1,300,0,25311,34.014279,-118.492679,2,0.0396,2021-10-21,POINT (139222.482 -443476.358),LINESTRING (139250.6453259104 -443504.64285780...
2,300,0,25311,34.01457,-118.493009,3,0.0849,2021-10-21,POINT (139191.491 -443444.567),LINESTRING (139250.6453259104 -443504.64285780...
3,300,0,25311,34.01477,-118.49325,4,0.116,2021-10-21,POINT (139168.881 -443422.739),LINESTRING (139250.6453259104 -443504.64285780...
4,300,0,25311,34.015089,-118.493629,5,0.1655,2021-10-21,POINT (139133.315 -443387.915),LINESTRING (139250.6453259104 -443504.64285780...
...,...,...,...,...,...,...,...,...,...,...
22213,300,0,25403,34.00871,-118.49001,329,10.7761,2021-10-21,POINT (139478.819 -444090.091),LINESTRING (133982.2906116628 -440495.08721612...
22214,300,0,25403,34.009059,-118.490109,330,10.8161,2021-10-21,POINT (139469.058 -444051.529),LINESTRING (133982.2906116628 -440495.08721612...
22215,300,0,25403,34.00936,-118.49019,331,10.8508,2021-10-21,POINT (139461.045 -444018.265),LINESTRING (133982.2906116628 -440495.08721612...
22216,300,0,25403,34.00963,-118.490299,332,10.8825,2021-10-21,POINT (139450.501 -443988.480),LINESTRING (133982.2906116628 -440495.08721612...
